# Using Langgraph for authentication

In [1]:
from typing import TypedDict,Optional
from langgraph.graph import StateGraph
from langgraph.graph import END

**State:**
It is like a dictionnary or an object which is mutable. State contains all informations which are passed through our graph. Think of *State* like the global context . State params are optional. In other words, they can either hold some value or not.

**Let's create a state class**

In [2]:
# State class
class AuthState(TypedDict):
    username:Optional[str]
    password:Optional[str]
    isAuthenticated:Optional[bool]

In [3]:
UserOne:AuthState = {
    "username":"barack adma",
    "password":"password",
    "isAuthenticated":False
}
UserTwo:AuthState = {
    "username":"ido",
    "password":"jiji",
    "isAuthenticated":False
}
print(UserOne)

{'username': 'barack adma', 'password': 'password', 'isAuthenticated': False}


# Nodes
It is a unit of our graph. We could even say it is the core part. A node handles some specifics or key tasks that our agent has to do. For instance , a node may retrieve some data from a database, another will sum up this informations and so on.


We can define a node which take some informations from our user

In [4]:
def inputNode(state):
    if state["username"] == "": # you could also do state.get("username",") == ""
        state["username"] = input("set your username")

    password= input("Set your password")
    return {"password":password}

inputNode(UserOne)

Set your password password2


{'password': 'password2'}

For validating ,we can use a validating node. For instance, if temperature can't be lower than 0 , we can throw an error to ask user to set another value

In [5]:
def validationNode(state):
    username = state.get("username","")
    password= state.get("password","")

    if username == "ido" and password == "jiji":
        isAuthenticated = True
    else:
        isAuthenticated = False

    return {"isAuthenticated":isAuthenticated}
    
validationNode(UserOne)

{'isAuthenticated': False}

## After validation , we have two branch: success or fail .We must add a node to both of them

In [6]:
def successNode(state):
    return {"output":"Validation succeeded"}

def failNode(state):
    return {"output":"Validation failed! Please try again"}

successNode(UserOne)

{'output': 'Validation succeeded'}

# Router Node

In [7]:
def routerNode(state):
    if state["isAuthenticated"]:
        return "successNode"
    else:
        return "failNode"

routerNode(UserOne)

'failNode'

We can also create succes or failure node ...
# Lets see how to add node to graph
We use  **StateGraph** for achieving this

In [8]:
workflow  = StateGraph(AuthState)
workflow

For adding a node to a graph , we must set:

*   node name
*   node function

In [9]:
workflow.add_node("inputNode",inputNode)

In [10]:
workflow.add_node("validationNode",validationNode)

In [11]:
workflow.add_node("success",successNode)

In [12]:
workflow.add_node("fail",failNode)

We can't define a node twice. It will throw an err. For instance if you write this lien:workflow.add_node("input_node",inputNode)
you'll get an error like :'input_node' is defined

**Edge**:Like it sounds, edge is the connection between nodes . It leads the agents during its transition from one node to another. To add an edge, it is quiete simple: we write workflow.add_edge("pastNode","nextNode")

In [13]:
workflow.add_edge("inputNode","validationNode")

To conclude the flow after agents has succeeded a task , we can put an edge between success node and END

In [14]:
workflow.add_edge("success", END)

In [15]:
workflow.add_edge("fail", "inputNode")

For enabling decision-making , we can use conditional_edges:It is like if else.
We must first and foremost define routerNode.
Params of add_conditional_edges:


*   start  
*   routerNode

*  conditions:Regarding to the condition state , we will target one the two nodes





In [16]:
workflow.add_conditional_edges("validationNode", routerNode, {
    "successNode": "success",
    "failNode": "fail"
})

**Entry point**:It is where we must start our graph

In [17]:
workflow.set_entry_point("inputNode")

In [18]:
app = workflow.compile()

Lets invoke our app with UserOne and see the result

In [19]:
result = app.invoke(UserTwo)
print(result)

Set your password jiji


{'username': 'ido', 'password': 'jiji', 'isAuthenticated': True}


Our Authentication succeeded. Actually , our authentication is too simple. My purpose lies on explaining what langgraph is , not validation process:-)